In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


In [2]:
images_train = np.load("../data/images/images_train.npy")
images_val = np.load("../data/images/images_val.npy")
images_test = np.load("../data/images/images_test.npy")
num_samples_train = images_train.shape[0]
num_samples_val = images_val.shape[0]
num_samples_test = images_test.shape[0]

images_train = images_train.reshape(num_samples_train, -1)
images_val = images_val.reshape(num_samples_val, -1)
images_test = images_test.reshape(num_samples_test, -1)
hog_train = np.load("../data/hog/hog_train.npy").reshape(num_samples_train, -1)
hog_val = np.load("../data/hog/hog_val.npy").reshape(num_samples_val, -1)
hog_test = np.load("../data/hog/hog_test.npy").reshape(num_samples_test, -1)
canny_edges_train  = np.load("../data/canny_edges/canny_edges_train.npy").reshape(num_samples_train, -1)
canny_edges_val  = np.load("../data/canny_edges/canny_edges_val.npy").reshape(num_samples_val, -1)
canny_edges_test  = np.load("../data/canny_edges/canny_edges_test.npy").reshape(num_samples_test, -1)
contours_train = np.load("../data/contours/contours_train.npy").reshape(num_samples_train, -1)
contours_val = np.load("../data/contours/contours_val.npy").reshape(num_samples_val, -1)
contours_test = np.load("../data/contours/contours_test.npy").reshape(num_samples_test, -1)


In [3]:
X_train = np.hstack((images_train, hog_train, canny_edges_train, contours_train))
X_val = np.hstack((images_val, hog_val, canny_edges_val, contours_val))
X_test = np.hstack((images_test, hog_test, canny_edges_test, contours_test))

y_train = np.load("../data/images/label_ids_train.npy")
y_val = np.load("../data/images/label_ids_val.npy")
y_test = np.load("../data/images/label_ids_test.npy")


In [4]:
labels_train = np.load("../data/images/labels_train.npy")
class_lookup = list(set(zip(y_train, labels_train)))
class_lookup = {id_value: class_value for id_value, class_value in class_lookup}
class_lookup


{2: 'greenstick',
 0: 'hairline',
 8: 'impacted',
 3: 'comminuted',
 9: 'avulsion',
 5: 'pathological',
 7: 'oblique',
 1: 'spiral',
 4: 'dislocation',
 6: 'longitudinal'}

In [5]:
model = LogisticRegression(max_iter=1000, multi_class='auto', solver='lbfgs')
model.fit(X_train, y_train)


/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto')

In [7]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


Accuracy: 0.21428571428571427
Classification Report:


,precision,recall,f1-score,support
hairline,0.000000,0.000000,0.000000,10.000000
spiral,0.142857,0.166667,0.153846,12.000000
greenstick,0.200000,0.375000,0.260870,16.000000
comminuted,0.384615,0.357143,0.370370,14.000000
dislocation,0.222222,0.315789,0.260870,19.000000
pathological,0.285714,0.222222,0.250000,18.000000
longitudinal,0.000000,0.000000,0.000000,12.000000
oblique,0.363636,0.250000,0.296296,16.000000
impacted,0.000000,0.000000,0.000000,9.000000
avulsion,0.272727,0.214286,0.240000,14.000000
